# 시위정보 시각화

## 1_크롤링 - 실시간 시위정보 데이터 가져와서 PDF로 저장하기

In [ ]:
#PC버전
!pip install selenium==4.1.5
!pip install webdriver_manager
from selenium import webdriver
import requests
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
service = Service(executable_path=ChromeDriverManager().install()) #크롬드라이버 설치
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service)

# 지정된 URL로 이동
driver.get('https://www.smpa.go.kr/user/nd54882.do')

# 페이지에서 이벤트 URL 추출
e_list = []
sel_01 = driver.find_elements(By.CLASS_NAME, 'subject')
for a in sel_01:
    event_date = a.text.split(" ")[-2]
    bn = a.find_element(By.TAG_NAME, 'a').get_attribute('href').split(",")[-1].split("'")[1]
    basic_url = f"https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo={bn}"
    e_list.append([event_date, basic_url])



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
e_list

[['241216',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314890'],
 ['241215',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314889'],
 ['241214',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314888'],
 ['241213',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314846'],
 ['241212',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314813'],
 ['241211',
  'https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo=00314790'],
 ['241210',
  'https://www.s

In [ ]:

# PDF 파일 다운로드 함수
def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, "wb") as file:
        file.write(response.content)


In [ ]:
# 이벤트 데이터 처리
import requests
e_list2 = []
for event_date, url in e_list:
    driver.get(url)
    pdf_url = driver.find_element(By.PARTIAL_LINK_TEXT, '.pdf').get_attribute('onclick').split("'")[-2]
    pdf_down_url = f"https://www.smpa.go.kr/common/attachfile/attachfileDownload.do?attachNo={pdf_url}"
    download_file(pdf_down_url, f"{event_date}_downloaded_file.pdf")
    e_list2.append([event_date,url, f"{event_date}_downloaded_file.pdf"])

## 2_가져온 데이터 PDF를 추출하여 STRUCTURED OUTPUT으로 구조화해서 저장하기

In [ ]:
# pdf 기반 assistant api 구현하기
# 1. openai 로그인하기
#client 생성
!pip install openai

import getpass
pw = getpass.getpass("openai API 키 : ")
from openai import OpenAI

client = OpenAI(
    api_key=pw,
)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


openai API 키 :  ········


In [ ]:
#2.파일별로 assistant를 만들어서 원하는 데이터 output 구축하기
for b in e_list2:
    print(b[2])

241216_downloaded_file.pdf
241215_downloaded_file.pdf
241214_downloaded_file.pdf
241213_downloaded_file.pdf
241212_downloaded_file.pdf
241211_downloaded_file.pdf
241210_downloaded_file.pdf
241209_downloaded_file.pdf
241208_downloaded_file.pdf
241207_downloaded_file.pdf


structured output 결과 스키마 미리구축해두기

In [ ]:
{"시위일자":"2024-12-06",
                    "시위명":"고등무상급식 촉구집회",
                    "시위장소":"서울시 광진구 구의동 611",
                    "시위인원":100,
                    "시위시간":{"시간":14,"분":0},
}

{'시위일자': '2024-12-06',
 '시위명': '고등무상급식 촉구집회',
 '시위장소': '서울시 광진구 구의동 611',
 '시위인원': 100,
 '시위시간': {'시간': 14, '분': 0}}

In [ ]:
my_wannabe_response={
  "name": "demonstration",
  "schema": {
    "type": "object",
    "properties": {
      "시위일자": {
        "type": "string",
        "description": "The date of the demonstration in YYYY-MM-DD format."
      },
      "시위명": {
        "type": "string",
        "description": "The name of the demonstration."
      },
      "시위장소": {
        "type": "string",
        "description": "The location where the demonstration will take place."
      },
      "시위인원": {
        "type": "number",
        "description": "The estimated number of participants in the demonstration."
      },
      "시위시간": {
        "type": "object",
        "properties": {
          "시간": {
            "type": "number",
            "description": "The hour of the demonstration."
          },
          "분": {
            "type": "number",
            "description": "The minute of the demonstration."
          }
        },
        "required": [
          "시간",
          "분"
        ],
        "additionalProperties": False
      }
    },
    "required": [
      "시위일자",
      "시위명",
      "시위장소",
      "시위인원",
      "시위시간"
    ],
    "additionalProperties": False
  },
  "strict": True
}

In [ ]:
response_format={
    "type": "json_schema",
    "json_schema": my_wannabe_response
}

In [ ]:
response_format

{'type': 'json_schema',
 'json_schema': {'name': 'demonstration',
  'schema': {'type': 'object',
   'properties': {'시위일자': {'type': 'string',
     'description': 'The date of the demonstration in YYYY-MM-DD format.'},
    '시위명': {'type': 'string', 'description': 'The name of the demonstration.'},
    '시위장소': {'type': 'string',
     'description': 'The location where the demonstration will take place.'},
    '시위인원': {'type': 'number',
     'description': 'The estimated number of participants in the demonstration.'},
    '시위시간': {'type': 'object',
     'properties': {'시간': {'type': 'number',
       'description': 'The hour of the demonstration.'},
      '분': {'type': 'number',
       'description': 'The minute of the demonstration.'}},
     'required': ['시간', '분'],
     'additionalProperties': False}},
   'required': ['시위일자', '시위명', '시위장소', '시위인원', '시위시간'],
   'additionalProperties': False},
  'strict': True}}

In [ ]:
# 독립 파일 한개를 올려보기
#파일업로드하기(오늘까지 원달러 환율데이터)


In [ ]:
my_file_name = e_list2[0][2]

#vector store 활용시
# vector_store = client.beta.vector_stores.create(name="시위 정보 벡터 스토어")
# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#     vector_store_id=vector_store.id,
#     files=[open(my_file_name, "rb")]
# )
#파일첨부 활용시
#생성된 test.xlsx 파일첨부
file = client.files.create(
  file=open(my_file_name, "rb"),
  purpose='assistants'
)

In [ ]:

#업로드된 파일로 assistant 생성
assistant = client.beta.assistants.create(
  name=f"PDF에서 시위정보를 찾아내는 봇_{my_file_name}",
  description="너는pdf의 내용을 보고 시위정보를 전체적으로 파악하는 봇이야.",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
  instructions="pdf안에 있는 데이터를 양식(날짜,시간,행사명,참여인원,주소)에 맞게 출력해",
)

In [ ]:
#thread 생성
thread = client.beta.threads.create()
print(thread)
print(thread.id)

Thread(id='thread_bRe7EyIHU2MKZqQVbFQs9pVw', created_at=1734160487, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
thread_bRe7EyIHU2MKZqQVbFQs9pVw


In [ ]:
file.id

'file-ELpQgayJ2vQ7WYE8tckhzY'

In [ ]:
#메시지를 thread에 추가하기
messages = client.beta.threads.messages.create(
    attachments=[
        {
            "file_id": file.id,
            "tools": [{"type": "file_search"}]  # Specify the tool to process the attachment
        }
    ],
    thread_id = thread.id,
    role = 'user',
    content = '''You are a helpful assistant designed to output json.\n
    pdf안에 있는 데이터를 읽고 json양식에 맞게 json으로 출력해.\n
    (중요! : 미사여구 다뺴고, 내가 필요한 데이터 양식만 답변줘)\n
    출력되는 데이터의 구조의 예시는 아래와 같아\n

    {"시위일자":"2024-12-06",
     "시위명":"고등무상급식 촉구집회",
     "시위장소":"서울시 광진구 구의동 611",
     "시위인원":100,
     "시위시작시간":"14시",
     "시위종료시간":"18시",
    }

    ''',
)
print(messages)

Message(id='msg_BvcUwvN3k8ZHpjbwfPOvoFsC', assistant_id=None, attachments=[Attachment(file_id='file-ELpQgayJ2vQ7WYE8tckhzY', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='You are a helpful assistant designed to output json.\n\n    pdf안에 있는 데이터를 읽고 json양식에 맞게 json으로 출력해.\n\n    (중요! : 미사여구 다뺴고, 내가 필요한 데이터 양식만 답변줘)\n\n    출력되는 데이터의 구조의 예시는 아래와 같아\n\n    \n    {"시위일자":"2024-12-06",\n     "시위명":"고등무상급식 촉구집회",\n     "시위장소":"서울시 광진구 구의동 611",\n     "시위인원":100,\n     "시위시작시간":"14시",\n     "시위종료시간":"18시",\n    }\n    \n    '), type='text')], created_at=1734160490, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_bRe7EyIHU2MKZqQVbFQs9pVw')


In [ ]:
#code
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    )
import time
time.sleep(20)

In [ ]:
run.status

'queued'

In [ ]:

#결과물 확인하기
#결과체크하기
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [ ]:
messages

SyncCursorPage[Message](data=[Message(id='msg_tJiMRC1RJ8QyOEPXGEah8TS2', assistant_id='asst_UnfcNjeQXrPzXobRs7f1xFRa', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='```json\n[\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "연세빌딩 앞 집회",\n        "시위장소": "남대문로5가",\n        "시위인원": 200,\n        "시위시작시간": "09:30",\n        "시위종료시간": "10:30"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "평강제일교회 앞 집회",\n        "시위장소": "오류동",\n        "시위인원": 500,\n        "시위시작시간": "10:00",\n        "시위종료시간": "11:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "국회 앞 집회",\n        "시위장소": "여의도동",\n        "시위인원": 250,\n        "시위시작시간": "13:00",\n        "시위종료시간": "15:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "예금보험공사 앞 집회",\n        "시위장소": "다동",\n        "시위인원": 700,\n        "시위시작시간": "14:00",\n        "시위종료시간": "15:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "동화면세점 앞 집회",

In [ ]:
my_text = messages.data[0].content[0].text.value

In [ ]:
my_text

'```json\n[\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "연세빌딩 앞 집회",\n        "시위장소": "남대문로5가",\n        "시위인원": 200,\n        "시위시작시간": "09:30",\n        "시위종료시간": "10:30"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "평강제일교회 앞 집회",\n        "시위장소": "오류동",\n        "시위인원": 500,\n        "시위시작시간": "10:00",\n        "시위종료시간": "11:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "국회 앞 집회",\n        "시위장소": "여의도동",\n        "시위인원": 250,\n        "시위시작시간": "13:00",\n        "시위종료시간": "15:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "예금보험공사 앞 집회",\n        "시위장소": "다동",\n        "시위인원": 700,\n        "시위시작시간": "14:00",\n        "시위종료시간": "15:00"\n    },\n    {\n        "시위일자": "2024-12-16",\n        "시위명": "동화면세점 앞 집회",\n        "시위장소": "세종로",\n        "시위인원": 10000,\n        "시위시작시간": "14:00",\n        "시위종료시간": "18:00"\n    }\n]\n```'

In [ ]:
print(my_text)

```json
[
    {
        "시위일자": "2024-12-16",
        "시위명": "연세빌딩 앞 집회",
        "시위장소": "남대문로5가",
        "시위인원": 200,
        "시위시작시간": "09:30",
        "시위종료시간": "10:30"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "평강제일교회 앞 집회",
        "시위장소": "오류동",
        "시위인원": 500,
        "시위시작시간": "10:00",
        "시위종료시간": "11:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "국회 앞 집회",
        "시위장소": "여의도동",
        "시위인원": 250,
        "시위시작시간": "13:00",
        "시위종료시간": "15:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "예금보험공사 앞 집회",
        "시위장소": "다동",
        "시위인원": 700,
        "시위시작시간": "14:00",
        "시위종료시간": "15:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "동화면세점 앞 집회",
        "시위장소": "세종로",
        "시위인원": 10000,
        "시위시작시간": "14:00",
        "시위종료시간": "18:00"
    }
]
```


In [ ]:
my_json_text = messages.data[0].content[0].text.value.split("json")[1].split("]")[0]+"]"
print(my_json_text)
import json
my_json = json.loads(my_json_text)


[
    {
        "시위일자": "2024-12-16",
        "시위명": "연세빌딩 앞 집회",
        "시위장소": "남대문로5가",
        "시위인원": 200,
        "시위시작시간": "09:30",
        "시위종료시간": "10:30"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "평강제일교회 앞 집회",
        "시위장소": "오류동",
        "시위인원": 500,
        "시위시작시간": "10:00",
        "시위종료시간": "11:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "국회 앞 집회",
        "시위장소": "여의도동",
        "시위인원": 250,
        "시위시작시간": "13:00",
        "시위종료시간": "15:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "예금보험공사 앞 집회",
        "시위장소": "다동",
        "시위인원": 700,
        "시위시작시간": "14:00",
        "시위종료시간": "15:00"
    },
    {
        "시위일자": "2024-12-16",
        "시위명": "동화면세점 앞 집회",
        "시위장소": "세종로",
        "시위인원": 10000,
        "시위시작시간": "14:00",
        "시위종료시간": "18:00"
    }
]


In [ ]:
import pandas as pd
pd.DataFrame(my_json)

시위일자          시위명    시위장소   시위인원 시위시작시간 시위종료시간
0  2024-12-16    연세빌딩 앞 집회  남대문로5가    200  09:30  10:30
1  2024-12-16  평강제일교회 앞 집회     오류동    500  10:00  11:00
2  2024-12-16      국회 앞 집회    여의도동    250  13:00  15:00
3  2024-12-16  예금보험공사 앞 집회      다동    700  14:00  15:00
4  2024-12-16   동화면세점 앞 집회     세종로  10000  14:00  18:00

In [ ]:
# 모든 PDF파일 작업시작
import os
pdf_file_list = []
for a in os.listdir():
    if ".pdf" in a:
        pdf_file_list.append(a)
pdf_file_list

['241207_downloaded_file.pdf',
 '241208_downloaded_file.pdf',
 '241209_downloaded_file.pdf',
 '241210_downloaded_file.pdf',
 '241211_downloaded_file.pdf',
 '241212_downloaded_file.pdf',
 '241213_downloaded_file.pdf',
 '241214_downloaded_file.pdf',
 '241215_downloaded_file.pdf',
 '241216_downloaded_file.pdf',
 'downloaded_file.pdf']

## 3_가져온 데이터를 pandas로 변경

In [ ]:
my_et_df = pd.DataFrame(['시위일자','시위명','시위장소','시위인원','시위시작시간','시위종료시간']).T
my_et_df.columns = my_et_df.loc[0]
# 기본 어시스턴트 제공 거기서 추가작업진행
for my_file_name in pdf_file_list:
    try:
        file = client.files.create(
          file=open(my_file_name, "rb"),
          purpose='assistants'
        )
        time.sleep(10)

        #메시지를 thread에 추가하기
        messages = client.beta.threads.messages.create(
            attachments=[
                {
                    "file_id": file.id,
                    "tools": [{"type": "file_search"}]  # Specify the tool to process the attachment
                }
            ],
            thread_id = thread.id,
            role = 'user',
            content = '''이번파일도 분석한뒤 답변해줘''',
        )
        #code
        run = client.beta.threads.runs.create(
            thread_id = thread.id,
            assistant_id = assistant.id,
            )
        import time
        time.sleep(20)
        messages = client.beta.threads.messages.list(
            thread_id = thread.id
        )
        my_text = messages.data[0].content[0].text.value
        my_json_text = messages.data[0].content[0].text.value.split("json")[1].split("]")[0]+"]"
        my_json = json.loads(my_json_text)
        import pandas as pd
        df = pd.DataFrame(my_json)
        # display(df)
        my_et_df = pd.concat([my_et_df,df],axis=0)
    except:
        print(f"오류발생_파일명_{my_file_name}")
my_et_df = my_et_df.iloc[1:,:]
my_et_df
df = my_et_df.copy()

오류발생_파일명_241214_downloaded_file.pdf
오류발생_파일명_241215_downloaded_file.pdf
오류발생_파일명_241216_downloaded_file.pdf
오류발생_파일명_downloaded_file.pdf


시위일자             시위명   시위장소    시위인원  시위시작시간  시위종료시간
0        시위일자             시위명   시위장소    시위인원  시위시작시간  시위종료시간
0  2024-12-13         여의도로 집회   여의도동   10000   13:00   15:00
1  2024-12-13   의사당역 1출 하위 집회  의사당대로   10000   13:00   15:00
2  2024-12-13        여의공원로 집회   여의도동   20000   13:30   14:30
3  2024-12-13   광복회관 앞~KBS 집회   여의대로    5000   13:30   15:00
4  2024-12-13         여의대로 집회   여의도동    5000   14:00   15:00
5  2024-12-13      동화면세점 앞 집회   세종대로   20000   15:00   20:00
6  2024-12-13         국회 앞 집회  의사당대로  200000   15:00      미정
7  2024-12-13   전쟁기념관 북문 앞 집회   한강대로     500   16:00   17:00
8  2024-12-13    더불어민주당사 앞 집회    여의도     200   16:00   21:00
9  2024-12-13  서울역광장→전쟁기념관 집회    봉래동    2000   17:00   20:00

In [ ]:
df

시위일자             시위명   시위장소    시위인원 시위시작시간 시위종료시간
0  2024-12-13         여의도로 집회   여의도동   10000  13:00  15:00
1  2024-12-13   의사당역 1출 하위 집회  의사당대로   10000  13:00  15:00
2  2024-12-13        여의공원로 집회   여의도동   20000  13:30  14:30
3  2024-12-13   광복회관 앞~KBS 집회   여의대로    5000  13:30  15:00
4  2024-12-13         여의대로 집회   여의도동    5000  14:00  15:00
5  2024-12-13      동화면세점 앞 집회   세종대로   20000  15:00  20:00
6  2024-12-13         국회 앞 집회  의사당대로  200000  15:00     미정
7  2024-12-13   전쟁기념관 북문 앞 집회   한강대로     500  16:00  17:00
8  2024-12-13    더불어민주당사 앞 집회    여의도     200  16:00  21:00
9  2024-12-13  서울역광장→전쟁기념관 집회    봉래동    2000  17:00  20:00

## 4_해당 주소를 위도/경도로 변경

In [ ]:
#기본 chatmode를 활용해서 주소를 정재하기
from openai import OpenAI
def Text_To_Address(my_address):
    response = client.chat.completions.create(
      model="gpt-4o-mini-2024-07-18",
      messages=[
        { "role": "system","content": "애매한 주소를 대한민국에 가장 명확한 주소를 찾아서 주소검색에 최적화된 명사를 대답해주는 봇이야."},
        { "role": "user","content": "연세빌딩 앞 집회"},
        { "role": "assistant","content": "서울 노원구 동일로178길 19-38 연세빌딩"},
        { "role": "user","content": "배00 의원 사무실 앞<삼전동>"},
        { "role": "assistant","content": "서울특별시 송파구 삼전동"},
        { "role": "user","content": """이화여대 정문 → 신촌 유플렉스
    ※행진경로(0.8km,1개 차로):이화여대 정문→신촌기차역
    →정가신촌직영점→신촌유플렉스
    <대현동 등>"""},
        { "role": "assistant","content": "이화여대정문"},
        { "role": "user","content": """연세로 양방 전차로 및 스타광장 內
    <창천동> """},
        { "role": "assistant","content": "연세로"},
        { "role": "user","content": """ 의사당역 5出 앞 3개차로 <여의도"""},
        { "role": "assistant","content": "국회의사당역 5번출구"},
        { "role": "user","content": """동화면세점~코리아나호텔 앞 편도 全차로
    <세종대로> """},
        { "role": "assistant","content": "동화면세점"},
        { "role": "user","content": """여의대로 광복회관 앞"""},
        { "role": "assistant","content": "여의대로 광복회관"},
        { "role": "user","content": """세종대로 동화면세점 앞"""},
        { "role": "assistant","content": "세종대로 동화면세점"},
        { "role": "user","content": """서울시 영등포구 여의도동 17-16 더불어민주당 사당ActivityResult-Lo.."""},
        { "role": "assistant","content": "서울시 영등포구 여의도동 17-16 더불어민주당"},
        { "role": "user","content": """서울특별시 중구 봉래동 서울역 광장 → 전쟁기념관"""},
        { "role": "assistant","content": "서울특별시 중구 봉래동 서울역"},
        { "role": "user","content": """의사당대로 국회 앞 집회"""},
        { "role": "assistant","content": "국회의사당"},
        { "role": "user","content": """서울역광장→전쟁기념관 집회"""},
        { "role": "assistant","content": "서울역"},
        { "role": "user","content": """전쟁기념관 북문 앞 집회"""},
        { "role": "assistant","content": "전쟁기념관"},
        { "role": "user","content": """의사당대로   국회 앞 집회"""},
        { "role": "assistant","content": "국회의사당"},
        { "role": "user","content": """여의대로 광복회관 앞~KBS 집회"""},
        { "role": "assistant","content": "여의대로 광복회관"},
        { "role": "user","content": my_address},

      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

In [ ]:
#테스트 Text_To_Address
Text_To_Address(""" 역사문화공원 앞
※행진경로(1.4km,인도):역사문화공원→도봉보건소R→
방학R→이안테라디움 앞
<창동 """)

'역사문화공원'

In [ ]:
df["NEW주소"] = (df['시위장소']+df['시위명']).apply(Text_To_Address)
df

C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_39152\4211500498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NEW주소"] = (df['시위장소']+df['시위명']).apply(Text_To_Address)


시위일자             시위명   시위장소    시위인원 시위시작시간 시위종료시간        NEW주소
0  2024-12-13         여의도로 집회   여의도동   10000  13:00  15:00         여의도동
1  2024-12-13   의사당역 1출 하위 집회  의사당대로   10000  13:00  15:00  국회의사당역 1번출구
2  2024-12-13        여의공원로 집회   여의도동   20000  13:30  14:30        여의도공원
3  2024-12-13   광복회관 앞~KBS 집회   여의대로    5000  13:30  15:00    여의대로 광복회관
4  2024-12-13         여의대로 집회   여의도동    5000  14:00  15:00         여의대로
5  2024-12-13      동화면세점 앞 집회   세종대로   20000  15:00  20:00   세종대로 동화면세점
6  2024-12-13         국회 앞 집회  의사당대로  200000  15:00     미정        국회의사당
7  2024-12-13   전쟁기념관 북문 앞 집회   한강대로     500  16:00  17:00     전쟁기념관 북문
8  2024-12-13    더불어민주당사 앞 집회    여의도     200  16:00  21:00  여의도 더불어민주당사
9  2024-12-13  서울역광장→전쟁기념관 집회    봉래동    2000  17:00  20:00        서울역광장

In [ ]:
#address to 위도경도
gmaps_api = getpass.getpass("구글맵API:")
!pip install googlemaps
import googlemaps
# API 키 입력
gmaps = googlemaps.Client(key=gmaps_api)
# 주소 입력
address = "서울특별시 성동구 한양대학교 본관"
# 주소를 위도와 경도로 변환
geocode_result = gmaps.geocode(address)
# 결과 출력
if geocode_result:
    location = geocode_result[0]['geometry']['location']
    print(f"위도: {location['lat']}, 경도: {location['lng']}")
else:
    print("주소를 찾을 수 없습니다.")
def address_to_latlng(address):
    gmaps = googlemaps.Client(key=gmaps_api)
    geocode_result = gmaps.geocode(address)
    return geocode_result[0]['geometry']['location']

df[['위도', '경도']] = df['NEW주소'].apply(lambda x: pd.Series(address_to_latlng(x)))



구글맵API: ········



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


위도: 37.5559817, 경도: 127.0447411


C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_39152\3891845753.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['위도', '경도']] = df['NEW주소'].apply(lambda x: pd.Series(address_to_latlng(x)))
C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_39152\3891845753.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['위도', '경도']] = df['NEW주소'].apply(lambda x: pd.Series(address_to_latlng(x)))


In [ ]:
df

시위일자             시위명   시위장소    시위인원 시위시작시간 시위종료시간        NEW주소  \
0  2024-12-13         여의도로 집회   여의도동   10000  13:00  15:00         여의도동   
1  2024-12-13   의사당역 1출 하위 집회  의사당대로   10000  13:00  15:00  국회의사당역 1번출구   
2  2024-12-13        여의공원로 집회   여의도동   20000  13:30  14:30        여의도공원   
3  2024-12-13   광복회관 앞~KBS 집회   여의대로    5000  13:30  15:00    여의대로 광복회관   
4  2024-12-13         여의대로 집회   여의도동    5000  14:00  15:00         여의대로   
5  2024-12-13      동화면세점 앞 집회   세종대로   20000  15:00  20:00   세종대로 동화면세점   
6  2024-12-13         국회 앞 집회  의사당대로  200000  15:00     미정        국회의사당   
7  2024-12-13   전쟁기념관 북문 앞 집회   한강대로     500  16:00  17:00     전쟁기념관 북문   
8  2024-12-13    더불어민주당사 앞 집회    여의도     200  16:00  21:00  여의도 더불어민주당사   
9  2024-12-13  서울역광장→전쟁기념관 집회    봉래동    2000  17:00  20:00        서울역광장   

          위도          경도  
0  37.529226  126.929383  
1  37.529239  126.917682  
2  37.525416  126.921516  
3  37.527150  126.916810  
4  37.524870  126.924314  
5  37.569635  126.976142  
6  37.531805  126.914155  
7  37.536606  126.977140  
8  37.529581  126.932680  
9  37.556074  126.971873

In [ ]:
data_prompt =f"""
df라는 변수에 판다스 데이터 프레임이 들어있어.
-여기에서 위도,경도를  기반으로 지도시각화를 해주는 folium 코드를 만들어줘.
-이떄, 시위일자/시위인원/ 시위시작시간~종료시간 도 함꼐 표시되게 해줘

columns:
{df.columns}
index:
{df.index}
shape:
{df.shape}
"""
print(data_prompt)


df라는 변수에 판다스 데이터 프레임이 들어있어. 
-여기에서 위도,경도를  기반으로 지도시각화를 해주는 folium 코드를 만들어줘.
-이떄, 시위일자/시위인원/ 시위시작시간~종료시간 도 함꼐 표시되게 해줘

columns:
Index(['시위일자', '시위명', '시위장소', '시위인원', '시위시작시간', '시위종료시간', 'NEW주소', '위도', '경도'], dtype='object')
index:
Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')
shape:
(10, 9)



## 5_위도/경도를 지도위에시각화

In [ ]:
!pip install folium
import folium
from folium.plugins import MarkerCluster

# 중심 좌표를 DataFrame의 위도와 경도의 평균으로 설정
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 마커 클러스터 추가
marker_cluster = MarkerCluster().add_to(m)

# 데이터프레임의 각 행을 반복하며 마커 추가
for _, row in df.iterrows():
    popup_text = f"""
    <b>시위일자:</b> {row['시위일자']}<br>
    <b>시위명:</b> {row['시위명']}<br>
    <b>시위장소:</b> {row['시위장소']}<br>
    <b>시위인원:</b> {row['시위인원']}명<br>
    <b>시간:</b> {row['시위시작시간']} ~ {row['시위종료시간']}
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(marker_cluster)

# 지도 저장 또는 표시
m.save("protest_map.html")  # 파일로 저장
m  # 주피터 노트북에서는 이 코드를 실행하면 지도가 표시됩니다.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install matplotlib
import folium
from folium.plugins import MarkerCluster
import matplotlib.colors as mcolors
import numpy as np

# 중심 좌표를 DataFrame의 위도와 경도의 평균으로 설정
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 마커 클러스터 추가
marker_cluster = MarkerCluster().add_to(m)

# 인원 수에 따른 색상 매핑 (적을수록 파란색, 많을수록 빨간색)
min_people = df['시위인원'].min()
max_people = df['시위인원'].max()

# 색상 스케일 설정
norm = mcolors.Normalize(vmin=min_people, vmax=max_people)
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red", ["blue", "red"])

# 데이터프레임의 각 행을 반복하며 원형 마커 추가
for _, row in df.iterrows():
    # 인원 수에 따른 색상 계산
    color = mcolors.to_hex(cmap(norm(row['시위인원'])))

    # 팝업 텍스트
    popup_text = f"""
    <b>시위일자:</b> {row['시위일자']}<br>
    <b>시위명:</b> {row['시위명']}<br>
    <b>시위장소:</b> {row['시위장소']}<br>
    <b>시위인원:</b> {row['시위인원']}명<br>
    <b>시간:</b> {row['시위시작시간']} ~ {row['시위종료시간']}
    """

    # 원형 마커 추가
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.5,
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['NEW주소']
    ).add_to(marker_cluster)

# 지도 저장 또는 표시
m.save("protest_map.html")  # 파일로 저장
m  # 주피터 노트북에서는 이 코드를 실행하면 지도가 표시됩니다.


  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 8.0/8.0 MB 55.5 MB/s eta 0:00:00
Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 42.3 MB/s eta 0:00:00
Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl (55 kB)
Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl (2.6 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 최종통합코드

In [ ]:
#PC버전

import getpass
pw = getpass.getpass("openai API 키 : ")
#address to 위도경도
gmaps_api = getpass.getpass("구글맵API:")

!pip install selenium==4.1.5
!pip install webdriver_manager
from selenium import webdriver
import requests
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
service = Service(executable_path=ChromeDriverManager().install()) #크롬드라이버 설치
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service)

# 지정된 URL로 이동
driver.get('https://www.smpa.go.kr/user/nd54882.do')

# 페이지에서 이벤트 URL 추출
e_list = []
sel_01 = driver.find_elements(By.CLASS_NAME, 'subject')
for a in sel_01:
    event_date = a.text.split(" ")[-2]
    bn = a.find_element(By.TAG_NAME, 'a').get_attribute('href').split(",")[-1].split("'")[1]
    basic_url = f"https://www.smpa.go.kr/user/nd54882.do?View&uQ=&pageST=SUBJECT&pageSV=&imsi=imsi&page=1&pageSC=SORT_ORDER&pageSO=DESC&dmlType=&boardNo={bn}"
    e_list.append([event_date, basic_url])

# PDF 파일 다운로드 함수
def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, "wb") as file:
        file.write(response.content)
# 이벤트 데이터 처리
import requests
e_list2 = []
for event_date, url in e_list:
    driver.get(url)
    pdf_url = driver.find_element(By.PARTIAL_LINK_TEXT, '.pdf').get_attribute('onclick').split("'")[-2]
    pdf_down_url = f"https://www.smpa.go.kr/common/attachfile/attachfileDownload.do?attachNo={pdf_url}"
    download_file(pdf_down_url, f"{event_date}_downloaded_file.pdf")
    e_list2.append([event_date,url, f"{event_date}_downloaded_file.pdf"])

# pdf 기반 assistant api 구현하기
# 1. openai 로그인하기
#client 생성
!pip install openai


from openai import OpenAI

client = OpenAI(
    api_key=pw,
)

my_file_name = e_list2[0][2]

file = client.files.create(
  file=open(my_file_name, "rb"),
  purpose='assistants'
)


#업로드된 파일로 assistant 생성
assistant = client.beta.assistants.create(
  name=f"PDF에서 시위정보를 찾아내는 봇_{my_file_name}",
  description="너는pdf의 내용을 보고 시위정보를 전체적으로 파악하는 봇이야.",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
  instructions="pdf안에 있는 데이터를 양식(날짜,시간,행사명,참여인원,주소)에 맞게 출력해",
)

#thread 생성
thread = client.beta.threads.create()
print(thread)
print(thread.id)

#메시지를 thread에 추가하기
messages = client.beta.threads.messages.create(
    attachments=[
        {
            "file_id": file.id,
            "tools": [{"type": "file_search"}]  # Specify the tool to process the attachment
        }
    ],
    thread_id = thread.id,
    role = 'user',
    content = '''You are a helpful assistant designed to output json.\n
    pdf안에 있는 데이터를 읽고 json양식에 맞게 json으로 출력해.\n
    (중요! : 미사여구 다뺴고, 내가 필요한 데이터 양식만 답변줘)\n
    출력되는 데이터의 구조의 예시는 아래와 같아\n

    {"시위일자":"2024-12-06",
     "시위명":"고등무상급식 촉구집회",
     "시위장소":"서울시 광진구 구의동 611",
     "시위인원":100,
     "시위시작시간":"14시",
     "시위종료시간":"18시",
    }

    ''',
)
print(messages)

#code
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    )
import time
time.sleep(20)

#결과물 확인하기
#결과체크하기
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

my_text = messages.data[0].content[0].text.value
my_json_text = messages.data[0].content[0].text.value.split("json")[1].split("]")[0]+"]"
print(my_json_text)
import json
my_json = json.loads(my_json_text)
import pandas as pd
display(pd.DataFrame(my_json))


# 모든 PDF파일 작업시작
import os
pdf_file_list = []
for a in os.listdir():
    if ".pdf" in a:
        pdf_file_list.append(a)
pdf_file_list

my_et_df = pd.DataFrame(['시위일자','시위명','시위장소','시위인원','시위시작시간','시위종료시간']).T
my_et_df.columns = my_et_df.loc[0]
# 기본 어시스턴트 제공 거기서 추가작업진행
for my_file_name in pdf_file_list:
    try:
        file = client.files.create(
          file=open(my_file_name, "rb"),
          purpose='assistants'
        )
        #메시지를 thread에 추가하기
        messages = client.beta.threads.messages.create(
            attachments=[
                {
                    "file_id": file.id,
                    "tools": [{"type": "file_search"}]  # Specify the tool to process the attachment
                }
            ],
            thread_id = thread.id,
            role = 'user',
            content = '''이번파일도 분석한뒤 답변해줘''',
        )
        #code
        run = client.beta.threads.runs.create(
            thread_id = thread.id,
            assistant_id = assistant.id,
            )
        import time
        time.sleep(20)
        messages = client.beta.threads.messages.list(
            thread_id = thread.id
        )
        my_text = messages.data[0].content[0].text.value
        my_json_text = messages.data[0].content[0].text.value.split("json")[1].split("]")[0]+"]"
        my_json = json.loads(my_json_text)
        import pandas as pd
        df = pd.DataFrame(my_json)
        # display(df)
        df = pd.concat([my_et_df,df],axis=0)
    except:
        print(f"오류발생_파일명_{my_file_name}")
my_et_df = my_et_df.iloc[1:,:]
my_et_df
df = my_et_df.copy()

#기본 chatmode를 활용해서 주소를 정재하기
from openai import OpenAI
def Text_To_Address(my_address):
    response = client.chat.completions.create(
      model="gpt-4o-mini-2024-07-18",
      messages=[
        { "role": "system","content": "애매한 주소를 대한민국에 가장 명확한 주소를 찾아서 주소검색에 최적화된 명사를 대답해주는 봇이야."},
        { "role": "user","content": "연세빌딩 앞 집회"},
        { "role": "assistant","content": "서울 노원구 동일로178길 19-38 연세빌딩"},
        { "role": "user","content": "배00 의원 사무실 앞<삼전동>"},
        { "role": "assistant","content": "서울특별시 송파구 삼전동"},
        { "role": "user","content": """이화여대 정문 → 신촌 유플렉스
    ※행진경로(0.8km,1개 차로):이화여대 정문→신촌기차역
    →정가신촌직영점→신촌유플렉스
    <대현동 등>"""},
        { "role": "assistant","content": "이화여대정문"},
        { "role": "user","content": """연세로 양방 전차로 및 스타광장 內
    <창천동> """},
        { "role": "assistant","content": "연세로"},
        { "role": "user","content": """ 의사당역 5出 앞 3개차로 <여의도"""},
        { "role": "assistant","content": "국회의사당역 5번출구"},
        { "role": "user","content": """동화면세점~코리아나호텔 앞 편도 全차로
    <세종대로> """},
        { "role": "assistant","content": "동화면세점"},
        { "role": "user","content": """여의대로 광복회관 앞"""},
        { "role": "assistant","content": "여의대로 광복회관"},
        { "role": "user","content": """세종대로 동화면세점 앞"""},
        { "role": "assistant","content": "세종대로 동화면세점"},
        { "role": "user","content": """서울시 영등포구 여의도동 17-16 더불어민주당 사당ActivityResult-Lo.."""},
        { "role": "assistant","content": "서울시 영등포구 여의도동 17-16 더불어민주당"},
        { "role": "user","content": """서울특별시 중구 봉래동 서울역 광장 → 전쟁기념관"""},
        { "role": "assistant","content": "서울특별시 중구 봉래동 서울역"},
        { "role": "user","content": """의사당대로 국회 앞 집회"""},
        { "role": "assistant","content": "국회의사당"},
        { "role": "user","content": """서울역광장→전쟁기념관 집회"""},
        { "role": "assistant","content": "서울역"},
        { "role": "user","content": """전쟁기념관 북문 앞 집회"""},
        { "role": "assistant","content": "전쟁기념관"},
        { "role": "user","content": """의사당대로   국회 앞 집회"""},
        { "role": "assistant","content": "국회의사당"},
        { "role": "user","content": """여의대로 광복회관 앞~KBS 집회"""},
        { "role": "assistant","content": "여의대로 광복회관"},
        { "role": "user","content": my_address},

      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content
df["NEW주소"] = (df['시위장소']+df['시위명']).apply(Text_To_Address)
df


!pip install googlemaps
import googlemaps
# API 키 입력
gmaps = googlemaps.Client(key=gmaps_api)
# 주소 입력
address = "서울특별시 성동구 한양대학교 본관"
# 주소를 위도와 경도로 변환
geocode_result = gmaps.geocode(address)
# 결과 출력
if geocode_result:
    location = geocode_result[0]['geometry']['location']
    print(f"위도: {location['lat']}, 경도: {location['lng']}")
else:
    print("주소를 찾을 수 없습니다.")
def address_to_latlng(address):
    try:
        gmaps = googlemaps.Client(key=gmaps_api)
        geocode_result = gmaps.geocode(address)
        result = geocode_result[0]['geometry']['location']
    except:
        result = {'lat': 37.5559817, 'lng': 127.0447411} #임의위치
    return result

df[['위도', '경도']] = df['NEW주소'].apply(lambda x: pd.Series(address_to_latlng(x)))



!pip install matplotlib
import folium
from folium.plugins import MarkerCluster
import matplotlib.colors as mcolors
import numpy as np

# 중심 좌표를 DataFrame의 위도와 경도의 평균으로 설정
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 마커 클러스터 추가
marker_cluster = MarkerCluster().add_to(m)

# 인원 수에 따른 색상 매핑 (적을수록 파란색, 많을수록 빨간색)
min_people = df['시위인원'].min()
max_people = df['시위인원'].max()

# 색상 스케일 설정
norm = mcolors.Normalize(vmin=min_people, vmax=max_people)
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red", ["blue", "red"])

# 데이터프레임의 각 행을 반복하며 원형 마커 추가
for _, row in df.iterrows():
    # 인원 수에 따른 색상 계산
    color = mcolors.to_hex(cmap(norm(row['시위인원'])))

    # 팝업 텍스트
    popup_text = f"""
    <b>시위일자:</b> {row['시위일자']}<br>
    <b>시위명:</b> {row['시위명']}<br>
    <b>시위장소:</b> {row['시위장소']}<br>
    <b>시위인원:</b> {row['시위인원']}명<br>
    <b>시간:</b> {row['시위시작시간']} ~ {row['시위종료시간']}
    """

    # 원형 마커 추가
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.5,
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['NEW주소']
    ).add_to(marker_cluster)

# 지도 저장 또는 표시
m.save("protest_map.html")  # 파일로 저장
m  # 주피터 노트북에서는 이 코드를 실행하면 지도가 표시됩니다.


KeyboardInterrupt: Interrupted by user